In [1]:
'''Code for fine-tuning Inception V3 for a new task.

Start with Inception V3 network, not including last fully connected layers.

Train a simple fully connected layer on top of these.


'''

import numpy as np
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Flatten, Dropout
import inception_v3 as inception

N_CLASSES = 3
IMSIZE = (299, 299)

# TO DO:: Replace these with paths to the downloaded data.
# Training directory
train_dir = '../data/sport3/train'
# Testing directory
test_dir = '../data/sport3/validation'


# Start with an Inception V3 model, not including the final softmax layer.
base_model = inception.InceptionV3(weights='imagenet')
print ('Loaded Inception model')

Using TensorFlow backend.


Loaded Inception model


In [2]:
for layer in base_model.layers:
    print(layer.output)

Tensor("input_1:0", shape=(?, 299, 299, 3), dtype=float32)
Tensor("Relu:0", shape=(?, 149, 149, 32), dtype=float32)
Tensor("cond/Merge:0", shape=(?, 149, 149, 32), dtype=float32)
Tensor("Relu_1:0", shape=(?, 147, 147, 32), dtype=float32)
Tensor("cond_1/Merge:0", shape=(?, 147, 147, 32), dtype=float32)
Tensor("Relu_2:0", shape=(?, 147, 147, 64), dtype=float32)
Tensor("cond_2/Merge:0", shape=(?, 147, 147, 64), dtype=float32)
Tensor("MaxPool:0", shape=(?, 73, 73, 64), dtype=float32)
Tensor("Relu_3:0", shape=(?, 73, 73, 80), dtype=float32)
Tensor("cond_3/Merge:0", shape=(?, 73, 73, 80), dtype=float32)
Tensor("Relu_4:0", shape=(?, 71, 71, 192), dtype=float32)
Tensor("cond_4/Merge:0", shape=(?, 71, 71, 192), dtype=float32)
Tensor("MaxPool_1:0", shape=(?, 35, 35, 192), dtype=float32)
Tensor("Relu_8:0", shape=(?, 35, 35, 64), dtype=float32)
Tensor("cond_8/Merge:0", shape=(?, 35, 35, 64), dtype=float32)
Tensor("Relu_6:0", shape=(?, 35, 35, 48), dtype=float32)
Tensor("Relu_9:0", shape=(?, 35, 35

In [3]:
base_model.get_layer('flatten').input

<tf.Tensor 'AvgPool_10:0' shape=(?, 1, 1, 2048) dtype=float32>

In [4]:
# Turn off training on base model layers
for layer in base_model.layers:
    layer.trainable = False
k=base_model.get_layer('flatten').output
# Add on new fully connected layers for the output classes.
x = Dense(32, activation='relu')(base_model.get_layer('flatten').output)
x = Dropout(0.5)(x)
predictions = Dense(N_CLASSES, activation='softmax', name='predictions')(x)

model = Model(input=base_model.input, output=predictions)

model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])


# Show some debug output
print (model.summary())

print ('Trainable weights')
print (model.trainable_weights)

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 299, 299, 3)   0                                            
____________________________________________________________________________________________________
convolution2d_1 (Convolution2D)  (None, 149, 149, 32)  0           input_1[0][0]                    
____________________________________________________________________________________________________
batchnormalization_1 (BatchNormal(None, 149, 149, 32)  0           convolution2d_1[0][0]            
____________________________________________________________________________________________________
convolution2d_2 (Convolution2D)  (None, 147, 147, 32)  0           batchnormalization_1[0][0]       
___________________________________________________________________________________________

In [6]:
# Data generators for feeding training/testing images to the model.
train_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory(
        train_dir,  # this is the target directory
        target_size=IMSIZE,  # all images will be resized to 299x299 Inception V3 input
        batch_size=32,
        class_mode='categorical')

test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
        test_dir,  # this is the target directory
        target_size=IMSIZE,  # all images will be resized to 299x299 Inception V3 input
        batch_size=32,
        class_mode='categorical')

model.fit_generator(
        train_generator,
        samples_per_epoch=32,
        nb_epoch=5,
        validation_data=test_generator,
        verbose=2,
        nb_val_samples=80)
model.save_weights('sport3_pretrain.h5')  # always save your weights after training or during training




img_path = '../data/sport3/validation/hockey/img_2997.jpg'
img = image.load_img(img_path, target_size=IMSIZE)
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)

x = inception.preprocess_input(x)

preds = model.predict(x)
print('Predicted:', preds)

Found 2999 images belonging to 3 classes.
Found 2898 images belonging to 3 classes.
Epoch 1/5
39s - loss: 1.2916 - acc: 0.3438 - val_loss: 1.0810 - val_acc: 0.4375
Epoch 2/5
36s - loss: 1.1192 - acc: 0.5938 - val_loss: 1.1320 - val_acc: 0.3750
Epoch 3/5
36s - loss: 1.1376 - acc: 0.4062 - val_loss: 1.0872 - val_acc: 0.4062
Epoch 4/5
36s - loss: 1.0743 - acc: 0.4688 - val_loss: 0.9573 - val_acc: 0.6042
Epoch 5/5
37s - loss: 1.1588 - acc: 0.4688 - val_loss: 0.9890 - val_acc: 0.4479
Predicted: [[ 0.15882917  0.65554577  0.18562511]]
